__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

6619


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

6619


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
1212,2499307974,"Участок 4,4 сот. (ИЖС)",2600000,,садовое товарищество Водолаз,"Продам земельный участок в районе Т-образного перекрестка. Земля оформлена, получен кадастровый номер участка. Чистая продажа. Огорожен, рядом живут соседи на постоянной основе. На участке есть емкость для воды, плодовые посадки: Грецкий орех, инжир, грана",3,2022-09-12 13:51:16.745,sevastopol,4.40,True,2.60,0.59,4-8
2822,2401918024,"Участок 7,7 сот. (ИЖС)",4800000,,ул. Василия Колодкина,"Продам замечательный ровный участок в районе Лабораторного шоссе. \nКатегория земли Ижс! \nУчасток расположен в 30 метрах от асфальтированной дороги, к самому участку подходит ровная грунтовая дорога. \nВ шаговой доступности (5 минут пешком) остановка Виктори",13,2022-09-13 13:56:54.600,sevastopol,7.70,True,4.80,0.62,4-8
3804,2149056727,Участок 4 сот. (ИЖС),1300000,,район Северная Сторона,"Севастополь, а в особенности его Северную сторону, абсолютно не затапливает ни при каких ливнях! Не зря расположение аэродрома «Бельбек» выбрано именно на Северной стороне. Смело переезжайте к нам! \n\nУчасток, 800 м от моря. Пляж Учкуевка, Северная сторона",11,2022-09-15 10:31:26.039,sevastopol,4.00,True,1.30,0.33,2-4


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6619 entries, 0 to 6618
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            6619 non-null   int64         
 1   title               6619 non-null   object        
 2   price               6619 non-null   int64         
 3   obj_name            6619 non-null   object        
 4   adr                 6619 non-null   object        
 5   description         6619 non-null   object        
 6   avito_page          6619 non-null   int64         
 7   ts                  6619 non-null   datetime64[ns]
 8   place               6619 non-null   object        
 9   area                6619 non-null   float64       
 10  is_IJS              6619 non-null   bool          
 11  priceM              6619 non-null   float64       
 12  priceMU             6619 non-null   float64       
 13  area_size_category  6619 non-null   category    

---

In [14]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

6619


## обновляем таблицу адресов

In [15]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [16]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [17]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/6619 [00:00<?, ?it/s]

  0%|          | 0/6619 [00:00<?, ?it/s]

In [18]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
2633,с. Орловка,"Севастополь,с. Орловка"
4058,"с. Орлиное, Благодатная ул.","Севастополь,с. Орлиное, Благодатная ул."
286,"Севастополь, Гераклейский бульвар, 12А","Севастополь, Гераклейский бульвар, 12А"
5834,"с. Вишнёвое, ул. Дружбы, 44","Севастополь,с. Вишнёвое, ул. Дружбы, 44"
3984,"Балаклава, Мускатная ул., 41","Севастополь,Балаклава, Мускатная ул., 41"
5556,Парковая ул.,"Севастополь,Парковая ул."
2938,"Предместная ул., 51","Севастополь,Предместная ул., 51"
3313,мыс Фиолент,"Севастополь,мыс Фиолент"
2017,"ул. Второй Обороны, 128","Севастополь,ул. Второй Обороны, 128"
1181,"пер. Бутакова, 8","Севастополь,пер. Бутакова, 8"


In [19]:
from lib.locator import GeocoderYandex
from yandex_keys import keys
locator = GeocoderYandex(keys[1])

# from lib.locator import GeocoderSimpleOSM
# locator = GeocoderSimpleOSM()

atr = AddressTransformer()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

# определяем координаты по строке адреса
loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')

loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 

[INFO    ] 2022-09-20 01:08:39 | LocationUpdater: 3307 addresses in index
[INFO    ] 2022-09-20 01:08:39 | LocationUpdater: 3926 addresses total
[INFO    ] 2022-09-20 01:08:39 | LocationUpdater: 3307 addresses defined
[INFO    ] 2022-09-20 01:08:39 | LocationUpdater: 619 addresses undefined


  0%|          | 0/619 [00:00<?, ?it/s]

[INFO    ] 2022-09-20 01:11:33 | LocationUpdater: 619 new addresses found


In [20]:
print( len(loc) )
loc.sample(3)

3926


,latitude,longitude,adr
2487,44.584879,33.466197,"Севастополь, район Гагаринский, Летчики, улица Колобова, 34/1"
101,44.54,33.44,"Севастополь,СНТ Муссон, 118"
1836,44.574132,33.464984,"Севастополь, улица Александра Маринеско, 1б/2"


In [21]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [22]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 6619
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6619 entries, 0 to 6618
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            6619 non-null   int64         
 1   title               6619 non-null   object        
 2   price               6619 non-null   int64         
 3   obj_name            6619 non-null   object        
 4   adr_orig            6619 non-null   object        
 5   description         6619 non-null   object        
 6   avito_page          6619 non-null   int64         
 7   ts                  6619 non-null   datetime64[ns]
 8   place               6619 non-null   object        
 9   area                6619 non-null   float64       
 10  is_IJS              6619 non-null   bool          
 11  priceM              6619 non-null   float64       
 12  priceMU             6619 non-null   float64       
 13  area